University of Michigan - ROB 201 Calculus for the Modern Engineer

---

# Julia HW06: Chapters 6.1-6.4 

### Topics Covered
- Path Length
- Root Finding via NLsolve
- Minima and Maxima are Stationary Points of Cost Functions
  - Without Equality Constraints
  - With Constraints
- Lagrangian Dynamics
- Optional Read: How to use JuMP for Constrained Minimization

<br>

<p align="center">
  <img src="data/HW06SpyDuck.png", width="70%">
</p>


# Problem 1: Mapping Signal Field with Respect to Signal Strength

### Scenario
As part of a terrain-mapping mission, your autonomous drone is following a path through a remote valley to map a radio signal field. The drone's **position (x, y)** at any given **signal strength level \( s \)** is tracked by three calibration functions: x(s), y(s), and z(s). Your task here is to calculate the total flight path length taken by the drone as it sweeps through the signal field from $s=0$ to $s=60$ decibels (dB).

**Hints:** 
  - You can use QuadGK to calculate integrals.
  - You can use ForwardDiff to compute derivatives.
  - The formula used to calculate the path length for 3D is identical to the one used for 2D.

<br>

<p align="center">
  <img src="data/HW06DroneFlightPath.png", width="50%">
</p>

Report your answer as `pathLength`

In [ ]:
# Define path functions
px(s) = 50 * exp(-0.01s) * cos(s / 8) + 0.3 * s^1.2
py(s) = 20 * log(s + 5) + 10 * sin(s / 6)
pz(s) = 5 * sin(s / 4) + 0.05 * s^2

# using Plots
# gr()
# Tf = 70
# # Sample points along the signal strength range
# s_vals = 0:0.5:Tf
# px_vals = [px(s) for s in s_vals]
# py_vals = [py(s) for s in s_vals]
# pz_vals = [pz(s) for s in s_vals]

# # Plot the path
# p1 = plot3d(px_vals, py_vals, pz_vals, legend=false, title="Drone Flight Path",
#     xlabel="X (km)", ylabel="Y (km)", zlabel="Z (km)", lw=3, color=:blue)
# scatter!([px(Tf)], [py(Tf)], [pz(Tf)],  color=:red, markershape=:star5, markersize=20)
# scatter!([px(0)], [py(0)], [pz(0)], color=:green, markershape=:circle, markersize=10)
# annotate!([px(Tf)-25], [py(Tf)], [pz(Tf)], "End"); annotate!([px(0)-25], [py(0)+5], [pz(0)], "Start")

# png(p1,"HW06DroneFlightPath")

In [ ]:
#
Tf = 70 # End of the flight
# pathLength = ?

###
### YOUR CODE HERE
### 

In [ ]:
#= Friendly Check =#
is_it_correct_check1 = abs(pathLength - 371.77483088242917) < 1e-3 ? "Yes" : "No"

@show is_it_correct_check1

In [ ]:
# ========== PROBLEM 1 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 1 BEGIN SOLUTION
using QuadGK
using ForwardDiff
vx(s) =  ForwardDiff.derivative(px, s)
vy(s) =  ForwardDiff.derivative(py, s)
vz(s) =  ForwardDiff.derivative(pz, s)
dL(s) = sqrt(( vx(s) )^2 + ( vy(s) )^2 + ( vz(s) )^2)
t_span = (0.0, Tf)
S, err = quadgk(dL, t_span[1], t_span[2])
pathLength = S
### 1 END SOLUTION 

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 1 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.

total_score_1 = is_it_correct_check1 == "Yes" ? 2 : 0

# Show score
println("Problem 1 Score: $total_score_1 / 2")

# Problem 2: Battery Health Check with Time-Varying Safety Threshold

### Scenario

As your Arctic robot roams across increasingly harsh terrain, not only does it consume power, but its **battery health deteriorates dynamically** due to temperature drops, internal resistance changes, and fluctuating computational demands.

To ensure safe operation, your robot is programmed to **return to base** not only when its battery hits zero, but also **when it drops below a time-varying safety threshold**, denoted as $E_{\text{safe}}(t)$. This threshold represents the minimum energy needed to safely return given expected terrain and system strain at time $t$. The units used are energy in watt-hour (Wh) and time in minutes.


🔋 Battery Energy Model:
```julia
function E(t)
    E₀ = 600.0  # Initial energy
    η = 0.98    # Cold-temperature efficiency factor

    if t < 15
        return E₀ - η * (12t + 40 * sin(t / 4))
    else
        return E(15) - η * (20 * (t - 15) + 60 * cos((t - 15) / 3))
    end
end
```

⚠️ Return Threshold Model $ E_{\text{safe}}(t) $

To model unpredictable conditions, we simulate $ E_{\text{safe}}(t) $ using a smooth random-like variation:
```julia
function Esafe(t)
    return 60 + 15 * sin(t / 3) + 10 * cos(t / 1.5)
end
```


In [ ]:
# Run me. I define some important functions

# Define the energy model function
function E(t)
    E₀ = 600.0  # Initial energy
    η = 0.98    # Cold-temperature efficiency factor

    if t <= 15
        return E₀ - η * (12t + 40 * sin(t / 4))
    else
        return E(15) - η * (20 * (t - 15) + 60 * cos((t - 15) / 3))
    end
end

# Define the threshold model function
function Esafe(t)
    return 60 + 15 * sin(t / 3) + 10 * cos(t / 1.5)
end

In [ ]:
# The Plot here is to give an idea of how the energy model and safety threshold evolve over time.
using Plots
ts = 0.0:0.5:50.0
E_vals = [E(t) for t in ts]
Esafe_vals = [Esafe(t) for t in ts]

# Plot
plot(ts, E_vals, label="Battery Energy E(t)", linewidth=2)
plot!(ts, Esafe_vals, label="Safety Threshold Esafe(t)", linewidth=2, linestyle=:dash)
xlabel!("Time (minutes)")
ylabel!("Energy (Wh)")
title!("Battery Energy vs Safety Return Threshold")

In [ ]:
# Solve for tStar when E(tStar) = Esafe(tStar)
# Report the battery's energy level at tStar
# Don't forget to call the package

# tStar = ??

# Estar = ??

###
### YOUR CODE HERE
### 

In [ ]:
#= Friendly Check =#
# if the value of is_it_correct_checkN is "Yes", then your answer is LIKELY correct. 
# If the value of is_it_correct_checkN is "No", then your answer is DEFINITELY wrong

is_it_correct_check2_1 = abs(tStar - 33.0694) < 1e-3 ? "Yes" : "No"
is_it_correct_check2_2 = abs(EStar - 35.0209) < 1e-3 ? "Yes" : "No"
is_it_correct_check2_3 = abs(myfun(tStar)) < 1e-3 ? "Yes" : "No"

@show is_it_correct_check2_1
@show is_it_correct_check2_2
@show is_it_correct_check2_3

In [ ]:
# ========== PROBLEM 2 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 2 BEGIN SOLUTION 
using NLsolve

myfun(t) = E(t) - Esafe(t)

# Define the function in a format suitable for NLsolve
function nlsolve_fun!(F, t)
    F[1] = myfun(t[1])
end

# Initial guess for t
initial_guess = [10.0]

# Solve the equation using NLsolve
solution = nlsolve(nlsolve_fun!, initial_guess)

# Extract the root
tStar = solution.zero[1]

# Find the battery's energy level
EStar = E(tStar)  # Call E(tStar) to ensure the function is evaluated

# Note that the last value should be close to zero
# If not, you may need to adjust the initial guess (should be greater than 10.0)
display([tStar EStar Esafe(tStar) myfun(tStar)] )   
### 2 END SOLUTION

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 2 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.

point_2A = is_it_correct_check2_1 == "Yes" ? 1 : 0
point_2B = is_it_correct_check2_2 == "Yes" && abs(Esafe(tStar) - EStar) < 1e-3 ? 1 : 0
point_2C = is_it_correct_check2_3 == "Yes" ? 1 : 0

total_score_2 = point_2A + point_2B + point_2C

# Show score
println("Problem 2 Score: $total_score_2 / 3")

<p align="center">
  <img src="data/HW06EvolutionaryFlagv01.png", width="70%">
</p>

# Problem 3: Mimimizing Joint Pain in Flag Bearing

### Scenario

Something that is equal parts scandalous and fictional: flag bearers in college marching bands develop shoulder, elbow, and wrist injuires at a higher rate than a typical college student. You, being a dual-major in Robotics and BME, have decided to do something about this. To provide some scientific gravitas to offset the major risk of your life-inexperience weighing against your application to do Human Subject Research (with the UM Institutional Review Board, aka IRB) you have developed a mechanical model of flag bearing. Your model has three links, with the shoulder serving as the base. Link 1 is the upper arm, link 2 is the forearem, and link 3 is the flag pole itself, gripped firmly in the hand. In the diagram below,  the angle of the wrist is captured by $\theta_3$, the angle of the elbow is $\theta_2$, and the angle of the shoulder is $\theta_1$, all measured in radians.

In addition to the length and mass of each link, your model takes into account the force of the wind blowing east-to-west against the flagpole at a steady 15 m/s. Because yes, marching bands perform outdoors!

Your **working hypothesis** is that joint positions that minimize the sum of the squared joint torques will maximize overall joint health. Importantly, your study will also take into account various constraints on how the flag is displayed to the crowd because, without any constraints, it does not take a genius to understand that effort might be minimized by dragging the flag on the ground, while simultaneoulsy obviating the overall purpose of the flag bearer being in the show. 

**An IRB** is concerned with protecting the welfare, rights, and privacy of human subjects. The IRB has the authority to approve, exempt, disapprove, monitor, and require modifications in all research activities that fall within its jurisdiction as specified by federal regulations and institutional policy. While our problem is tongue-in-cheek, IRBs are serious stuff!

<br>

<p align="center">
  <img src="data/HW06robotKinematicChain.png", width="70%">
</p>

### First Principles

Because the gradient is the transpose of the Jacobian, the linearization of $f:{\mathbb R}^n \to {\mathbb R}$ via the gradient can be written as 
$$
    f(x) \approx f(x_0) + \left( \nabla f(x_0) \right)^\top \cdot (x - x_0) = f(x_0) +\nabla f(x_0) \bullet \underbrace{(x - x_0)}_{\Delta x} = f(x_0) +\nabla f(x_0) \bullet \Delta x,\\
$$
where $u \bullet v$ denotes the **dot product or inner product** of the two $n$-vectors, $u$ and $v$. 
  - When $\nabla f(x_0) = 0_{n \times 1}$, then $x$ near $x_0$ $\implies f(x) \approx f(x_0)$ (aka, $\Delta x$ small $\implies f(x_0 + \Delta x) \approx f(x_0)$).
  - When $\nabla f(x_0) \neq 0_{n \times 1}$, then $\Delta x = - s \nabla f(x_0)$, $s > 0$ small guarantees that  $f(x_0 + \Delta x) \approx f(x_0) - s ||\nabla f(x_0) ||^2 < f(x_0)$.

### Definitions
  - $x_0 \in {\mathbb R}^n$ is a **stationary point** if $\nabla f(x_0)={0}_{n \times 1}$ (aka, a local extremum).
  
  - a stationary point $x_0 \in {\mathbb R}^n$ is a **local minimum** if $f(x_0) \le f(x)$ for all $x$ near $x_0$ (i.e., there exists $\delta >0$ such that for all $||x - x_0|| < \delta$, $f(x_0) \le f(x)$).
   
  - a stationary point $x_0 \in {\mathbb R}^n$ is a **local maximum** if $f(x_0) \ge f(x)$ for all $x$ near $x_0$ (i.e., there exists $\delta >0$ such that for all $||x - x_0|| < \delta$, $f(x_0) \ge f(x)$).
   
  - A stationary point $x_0$ that is **neither** a local minimum **nor** a local maximum is called a **saddle point**. In the special case of functions $f:{\mathbb R} \to {\mathbb R}$, saddle points are called **inflection points**. 


### Remark
Our objective here is to underline that roots of the gradient are candidate points for finding local minima and local maxima. In the end, while finding roots is relatively easy via NLsolve, it is not a very efficient way to find local minima because there are simply too many stationary points in a typical cost function. This is why Project 2 will focus on **Gradient Descent** as a more reliable means to minimize a cost function, even in the presence of equality constraints.

In [ ]:
# Run me: I find roots of the gradient, hence, I find stationary points!
#
using ForwardDiff, NLsolve, LinearAlgebra
using Plots
gr()  # Set GR as the backend for Plots
include("data/HW06JointTorque_utilities.jl")

# m/s wind speed, blowing from east to west
windX = -15.0 

# Compute torque required to hold flagpole and arm as specified by the 
# joint angles q1, q2, q3 = q in radians
tau(q) = jointTorques(q, windX)

# Define the cost function as sum of squares of the joint torques
cost(q) = sum(tau(q) .* tau(q))

# Define the gradient of the cost function in a form that is preferred by nlsolve
function grad_cost_roots!(F, q)
    F .= ForwardDiff.gradient(cost, q)
end


########################################
#  For Problem 3A Edit Below Here
########################################
#
# Initial joint angles. For your record, 
# our default is q0 = [90; 0; 0]*pi/180 
# 
q0 = [0; 90; -15]*pi/180 
#
########################################
#  For Problem 3A Edit Above Here
########################################

# Initial guess for NLsolve
initial_guess = q0

# Solve for the roots of the gradient using NLsolve
solution = nlsolve(grad_cost_roots!, initial_guess; iterations = Int(1e5))

# Extract the stationary points
q_stationary = solution.zero # joint angles
grad_cost_stationary = ForwardDiff.gradient(cost, q_stationary) # is the gradient zero?
tau_stationary = tau(q_stationary) # torques 


# Output the result
println("The cost at the initial joint angles is:    ", cost(q0))
println("The cost at the stationary joint angles is: ", cost(q_stationary))
println("\nThe gradient of the cost is: ", grad_cost_stationary, " \n")
println("Joint angles at the stationary point are:    ", q_stationary, "  radians")
println("Joint angles at the stationary point are: ", q_stationary*180/pi, "  degrees\n")
println("Torques at the stationary point are:", tau_stationary, "  Newtons")
println("The original torques were:          ", tau(q0), "  Newtons")

# Make a plot
fig = plot(; legend=:topleft, aspect_ratio=:equal)
plot_armFlagBearer(q_stationary)            
display(fig)


## P.3A: Report three distinct stationary points
Change `q0` in the above code cell and record `q_stationary` in the code cell below. You are looking for points where the gradient vanishes.

Give all answers in RADIANS. 

**Note:** One of your points can be the one we found for you.


In [ ]:
# Give all answers in radians

# q_stationary01 = 

# q_stationary02 = 

# q_stationary03 = 

###
### YOUR CODE HERE
###

In [ ]:
#= Friendly check =#

is_it_correct_check3a_1 = norm(q_stationary01-q_stationary02)*
norm(q_stationary02-q_stationary03)*norm(q_stationary03-q_stationary01) > 1e-5 ? "Yes" : "No"
if is_it_correct_check3a_1 == "Yes"
    println("Proposed stationary points are distinct")
else
    println("You seem to have used the same answer twice.")
end

grad_cost_stationary03 = ForwardDiff.gradient(cost, q_stationary03) # is the gradient zero?
is_it_correct_check3a_2 = norm(grad_cost_stationary03) < 1e-5 ? "Yes" : "No"
if is_it_correct_check3a_2 == "Yes"
    println("Gradient vanishes at your third proposed stationary point. 
        You can check the other points")
else
    println("Gradient does not vanish adequately at your third proposed stationary point. 
        Better check your other points too.")
end

## P.3B: Report three distinct TYPES of stationary points


<br>

<p align="center">
  <img src="data/HW06LocalMaxMinSaddleImages.png", width="80%">
</p>

<h6><a href="https://www.offconvex.org/2016/03/22/saddlepoints/" target="_blank">Image credit</a></h6>

<br>



<br>

<p align="center">
  <img src="data/HW06HessianTestForLocalMaxMin.png", width="80%">
</p>
<br>

```julia
# How to implement the above Proposition in Code
hessian_cost_stationary = ForwardDiff.hessian(cost, q_stationary)
F = eigen(hessian_cost_stationary)
F.values
```

Change `q0` in the code cell below and record `q_stationary` in the code cell that follows it. You are looking for points where the gradient vanishes AND the eigenvalues of the Hessian have special properties.

**Hint:** There is a very good chance that your prior answers for `q_stationary` contain at least two types of stationary points. You may even have all three already, but just did not know it.

In [ ]:
# Run me: I find roots of the gradient, hence, I find stationary points, 
# and I check their type
#
using ForwardDiff, NLsolve, LinearAlgebra
using Plots
gr()  # Set GR as the backend for Plots
include("data/HW06JointTorque_utilities.jl")

# Define the cost function as sum of squares of the joint torques
windX = -15.0 # m/s wind speed, blowing east to west
tau(q) = jointTorques(q, windX)
cost(q) = sum(tau(q) .* tau(q))

# Define the gradient of the cost function
function grad_cost_roots!(F, q)
    F .= ForwardDiff.gradient(cost, q)
end

########################################
#         Edit Below Here
########################################

# Initial joint angles
# Our default is q0 = [90; 0; 0]*pi/180 
# 
q0 = [0; 90; -15]*pi/180*pi/180 

########################################
#         Edit Above Here
########################################

# Initial guess for NLsolve
initial_guess = q0

# Solve for the roots of the gradient using NLsolve
solution = nlsolve(grad_cost_roots!, initial_guess; iterations = Int(1e5))

# Extract the stationary points
q_stationary = solution.zero # joint angles
grad_cost_stationary = ForwardDiff.gradient(cost, q_stationary) # is the gradient zero?
tau_stationary = tau(q_stationary) # torques 

# Evaluate the type of stationary point
hessian_cost_stationary = ForwardDiff.hessian(cost, q_stationary)
F = eigen(hessian_cost_stationary)
F.values

In [ ]:
# Record your answers here

# all eigenvalues are negative = n-dimensional downward pointing bowl
# q_localMax = 

# all eigenvalues are positive = n-dimensional upward pointing bowl
# q_localMin =

# eigenvalues are mixed or at least one is zero = either too flat or
# function is increasing in some directions and decreasing in others
# q_localSaddle =

###
### YOUR CODE HERE
###

In [ ]:
#= Friendly Check for a Saddle Point =#

# Collect the data
grad_cost_Saddle = ForwardDiff.gradient(cost, q_localSaddle)
hessian_cost_Saddle = ForwardDiff.hessian(cost, q_localSaddle)
F = eigen(hessian_cost_Saddle )
evaluesSaddle = F.values
minEvalue = minimum(evaluesSaddle)
maxEvalue = maximum(evaluesSaddle)
is_it_correct_check3b_3 = NaN # This will be set to "Yes" if the test passes

# Run the tests
if norm(grad_cost_Saddle) < 1e-5
    println("Passes the test for a Stationary Point")
    # Now check for local min, max, or saddle
    if maxEvalue < 0
        println("You found a Local Maximum and NOT a Saddle Point")
    elseif minEvalue > 0 
        println("You found a Local Minimum and NOT a Saddle Point")
    else
        println("You found a Local Saddle Point!")
        is_it_correct_check3b_3 = "Yes"
    end
else
    println("Fails the test for a Stationary Point")
end
    
# We will grade each stationary point type separately for 1/2 point each.    

# Dealing with Equality Constraints: the Method of Lagrange Multipliers


<br> The basic idea is to combine the orignal cost function with the equality constraints to form what is called a **Lagrangian**. Then, stationary points of the Lagrangian contain all of the local extremal of the cost that also satisfy the equality constraints. The idea is pure genious. It's implementation by hand is pure torture. It's implementation in code is cake. Read below for more technical details.  

<br>

<p align="center">
  <img src="data/HW06LagrangeMultipliers.png", width="90%">
</p>
<br>

### Example Code

In [ ]:
using ForwardDiff, NLsolve, LinearAlgebra, Plots
gr()  # Set GR as the backend for Plots

include("data/HW06JointTorque_utilities.jl")
windX = -15.0 # m/s blowing east to west
tau(q) = jointTorques(q, windX)
cost(q) = sum(tau(q) .* tau(q))

# Define two equality constraint functions
function constraint(q)
    positions = linkPostions(q)
    p3 = positions.p3  # Assume this function computes the position of the end of the flag pole
    # Place the flag 4.5 meters above the shoulder and one half meter in front of it
    # When designing your own constraints, you can more simply use linear combinations 
    # of q[i] set to zero. It's your choice. Experiment a bit.
    return [p3[2] - 4.5; q[2] - 0.5]
end

# qλ is a colum vector with the joint angles q followed by the Lagrange multipliers λ 

# Define the augmented cost function with the Lagrange multipliers
function Lagrangian(qλ)
    q = qλ[1:3]
    λ = qλ[4:end]
    return cost(q) + sum(λ .* constraint(q))
end

# Initial joint angles and Lagrange multiplier
q0 = [45; 0; 0]*pi/180
λ0 = [0.0; 0.0]

# Define the initial guess
initial_guess = [q0; λ0]

# Define the gradient of the augmented cost function
function gradLagrangianRoots!(F, qλ)
    F .= ForwardDiff.gradient(Lagrangian, qλ)
end

# Solve for the roots of the gradient using NLsolve
solution = nlsolve(gradLagrangianRoots!, initial_guess; iterations = Int(1e5))

# Extract the optimal joint angles and Lagrange multiplier
qλ_stationary = solution.zero
q_stationary = qλ_stationary[1:3]
λ_stationary = qλ_stationary[4:end]
tau_stationary = tau(q_stationary)
grad_cost_stationary = ForwardDiff.gradient(Lagrangian, qλ_stationary)

# Output the result
println("The Lagrangian at the initial joint angles is:    ", Lagrangian(initial_guess))
println("The Lagrangian at the stationary joint angles is: ", Lagrangian(qλ_stationary))
println("\nThe gradient of the Lagrangian is: ", grad_cost_stationary, "  radians\n")
println("Joint angles at the stationary point are: ", q_stationary, "  radians")
println("Joint angles at the stationary point are: ", q_stationary*180/pi, "  degrees\n")
println("Lagrange multipliers at the stationary point are: ", λ_stationary)
println("\nTorques at the stationary point are: ", tau_stationary, "  Newtons")
println("The original torques were:           ", tau(q0), "  Newtons")


# Initialize the plot
fig = plot(; legend=:topleft, aspect_ratio=:equal)
plot_armFlagBearer(q_stationary)
display(fig)

In [ ]:
# Run me, no points are at stake here
grad_Lagrangian = ForwardDiff.gradient(Lagrangian, qλ_stationary)
hessian_Lagrangian = ForwardDiff.hessian(cost, qλ_stationary)
F = eigen(hessian_Lagrangian)
evalues = F.values
@show minEvalue = minimum(evalues)
@show maxEvalue = maximum(evalues)
println("\nWith the method of Lagrange Multipliers, the stationary points are 
always saddle points of the Lagrangian. You cannot detect local minima via 
the Hessian. This is why in real engineering, we do not extensively use the 
Method of Lagrange Multipliers to handle constraints. We do use, however, a method 
that is suggested by the proof of the Method of Lagrange Multipliers. Proofs are useful.")  

## P.3C: Achieve an arm posture ROUGHLY similar to the robot

**Note:** Roughly means +/- twenty degrees for each joint, $\theta_i$.

<br>

<p align="center">
  <img src="data/HW06EvolutionaryFlagv02Outtake.png", style="transform: scaleX(-1)" width="20%">
</p>
<br>

### Remarks
- You may apply constraints directly to the joint angles (in radians).
- Use **no more than two constraints** total.
- For reference, a shoulder joint pointing straight down corresponds to an angle of $\frac{-\pi}{2}$.
- A configuration like the one shown below is perfectly acceptable.  
  _(Assume the flag is tilted 45 degrees to the right.)_
  
<br>

<table>
  <tr>
    <td><img src="data/HW06AccetableFlagSolution.png" width="200"></td>
  </tr>
</table>


In [ ]:
using ForwardDiff, NLsolve, LinearAlgebra, Plots
gr()  # Set GR as the backend for Plots

include("data/HW06JointTorque_utilities.jl")
windX = -15.0 # m/s blowing east to west
tau(q) = jointTorques(q, windX)
cost(q) = sum(tau(q) .* tau(q))

########################################
#         Edit Below Here
########################################


# Define two equality constraint functions
function constraint(q)
    #positions = linkPostions(q) # You do not really need these
    #p3 = positions.p3  # Computes the position of the end of the flag pole
    # Place the flag 3.0 meters above the shoulder and one half meter in front of it
    #return [p3[2] - 3.0; q[2] - 0.5]
    
    ###
    ### YOUR CODE HERE
    ###
    
    # If you want to change q0 (given below here), you can do that.
    # it will not matter that much when there is only one free variable
end

########################################
#         Edit Above Here
########################################


# qλ is a colum vector with the joint angles q followed by the Lagrange multipliers λ 

# Define the augmented cost function with the Lagrange multipliers
function Lagrangian(qλ)
    q = qλ[1:3]
    λ = qλ[4:end]
    return cost(q) + sum(λ .* constraint(q))
end

# Initial joint angles and Lagrange multiplier
q0 = [-90; -90; -90]*pi/180
λ0 = [0.0; 0.0]

# Define the initial guess
initial_guess = [q0; λ0]

# Define the gradient of the augmented cost function
function gradLagrangianRoots!(F, qλ)
    F .= ForwardDiff.gradient(Lagrangian, qλ)
end


# Solve for the roots of the gradient using NLsolve
solution = nlsolve(gradLagrangianRoots!, initial_guess; iterations = Int(1e5))

# Extract the optimal joint angles and Lagrange multiplier
qλ_stationary = solution.zero
q_stationary = qλ_stationary[1:3]
λ_stationary = qλ_stationary[4:end]
tau_stationary = tau(q_stationary)
grad_cost_stationary = ForwardDiff.gradient(Lagrangian, qλ_stationary)

# Output the result
println("The Lagrangian at the initial joint angles is:   ", Lagrangian(initial_guess))
println("The Lagrangian at the stationary joint angles is: ", Lagrangian(qλ_stationary))
println("\nThe gradient of the Lagrangian is: ", grad_cost_stationary, "  radians\n")
println("Joint angles at the stationary point are: ", q_stationary, "  radians")
println("Joint angles at the stationary point are: ", q_stationary*180/pi, "  degrees\n")
println("Lagrange multipliers at the stationary point are: ", λ_stationary)
println("\nTorques at the stationary point are: ", tau_stationary, "  Newtons")
println("The original torques were:           ", tau(q0), "  Newtons")


# Initialize the plot
fig = plot(; legend=:topleft, aspect_ratio=:equal)
plot_armFlagBearer(q_stationary)
display(fig)

In [ ]:
# Friendly check
is_it_correct_check3c_1 = length(constraint(q_stationary)) <= 2 ? "Yes" : "No"
if is_it_correct_check3c_1 == "No"
    println("Nope. Not allowed.")
end
q = q_stationary*180/pi

if (-110 < q[1] < -70)
    println("You are within 20 degree of the robot's shoulder angle.")
else
    println("You can eyeball the shoulder angle better than that!")
end

if (-110 < q[2] < -70)
    println("You are within 20 degree of the robot's elbow angle.")
else
    println("You can eyeball the elbow angle better than that!")
end
    
if (-155 < q[3] < -115)
    println("You are within 20 degree of the robot's flag pole angle.")
else
    println("You can eyeball the flag pole angle better than that!")
end

is_it_correct_check3c_2 = (-110 < q[1] < -70) && (-110 < q[2] < -70) && (-155 < q[3] < -115) ? "Yes" : "No"

println("\nIf you are unclear on the robot's nominal joint angles, feel free to debate them on Piazza.")

In [ ]:
# ========== PROBLEM 3 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 3A BEGIN SOLUTION 
q01 = [90; 0; 0]*pi/180
q_stationary01 = [0.9306967703453799, 0.18324529875803255, 0.07688786508513096]

q02 = [0.0, 0.0, 0.0] 
q_stationary02 = [0.0, 0.0, 0.0] 

q03 = [0; 90; -15]*pi/180
q_stationary03 = [-0.3010859742766978, 1.8650090772767822, -0.31788782115178776]
### 3A END SOLUTION 

### 3A BEGIN SOLUTION 
q01 = [90; 0; 0]*pi/180
q_localMin = [0.9306967703453799, 0.18324529875803255, 0.07688786508513096]

q02 = [0.0, 0.0, 0.0] 
q_localMax = [0.0, 0.0, 0.0] 

q03 = [0; 90; -15]*pi/180
q_localSaddle = [-0.3010859742766978, 1.8650090772767822, -0.31788782115178776]
### 3B END SOLUTION 

### 3C BEGIN SOLUTION 
return [q[1] + pi/2; q[2] + pi/2]
### 3C END SOLUTION 

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 3 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.

# 3A
point_3A = is_it_correct_check3a_1 == "Yes" && is_it_correct_check3a_2 == "Yes" && norm(ForwardDiff.gradient(cost, q_stationary01)) < 1e-4 && norm(ForwardDiff.gradient(cost, q_stationary02)) < 1e-4 ? 1 : 0

# 3B
grad_cost_1 = ForwardDiff.gradient(cost, q_localMin)
grad_cost_2 = ForwardDiff.gradient(cost, q_localMax)
hessian_cost_1 = ForwardDiff.hessian(cost, q_localMin)
hessian_cost_2 = ForwardDiff.hessian(cost, q_localMax)
F_1 = eigen(hessian_cost_1)
F_2 = eigen(hessian_cost_2)
minEvalue = minimum(F_1.values)
maxEvalue = maximum(F_2.values)

is_it_correct_check3b_1 = norm(grad_cost_1) < 1e-3 && minEvalue > 0 ? "Yes" : "No"
is_it_correct_check3b_2 = norm(grad_cost_2) < 1e-3 && maxEvalue < 0 ? "Yes" : "No"

point_3B_1 = is_it_correct_check3b_1 == "Yes" ? 0.5 : 0
point_3B_2 = is_it_correct_check3b_2 == "Yes" ? 0.5 : 0
point_3B_3 = is_it_correct_check3b_3 == "Yes" ? 0.5 : 0

# 3C
point_3C = is_it_correct_check3c_1 == "Yes" && is_it_correct_check3c_2 == "Yes" ? 1 : 0

total_score_3 = point_3A + point_3B_1 + point_3B_2 + point_3B_3 + point_3C

# Show score
println("Problem 3 Score: $total_score_3 / 3.5")

# Lagrangian Dynamics of a Slinky

After watching [SlinkyJosh: How This Guy Mastered the Slinky | Obsessed | WIRED](https://youtu.be/1eYAWlRCdUE), you’ve joined the ranks of those fascinated by this iconic toy from 1943. Naturally, as an analytically minded individual (is there any other kind?), you've decided to develop a **dynamical model** of the Slinky—convinced that understanding its mathematical behavior might just fast-track your slinkying skills.

Two defining properties of a Slinky are:
- It can **stretch and bend**.
- Each of its **98 coils has mass**.

As a warm-up exercise, you’ll model a simplified version of the Slinky under the following assumptions:

- The motion is constrained to a **2D plane** (no 3D modeling—for now).
- The Slinky is represented as a **series of point masses** (one for each coil), connected by alternating:
  - **linear (stretching) springs**, and  
  - **bending springs** (modeled as revolute joints with torsional springs).
- The structure follows this pattern:  
  `mass → stretching spring → mass → bending spring → mass → ...`  
  Each mass represents one coil of the Slinky.
- For this problem, you'll develop a model that captures the behavior of **two coils**.

<br>

> **Fun Fact:** The Slinky was invented in 1943 by **Richard James**, a naval engineer who discovered its unique behavior after accidentally knocking a tension spring off a shelf. It was first demonstrated at **Gimbels Department Store** in Philadelphia in 1945—and quickly became a hit.
> 
> **More Slinky Magic:**
> - [Slinky walking on a treadmill](https://youtu.be/ZJdZ4cbvyCg)  
> - [Slinky walking down stairs](https://youtu.be/yNlOfDHyaXg)


<br>

<p align="center">
  <img src="data/HW06TikZSlinkyModel.png", width="50%">
</p>
<br>

# Problem 4: Compute the Potential and Kinetic Energy for Two Coils of a Slinky

In this problem, you will compute the **potential** and **kinetic energy** of a simplified Slinky model consisting of **two coils**. 
  - p0, p1, p2, p3 = $p_0$, $p_1$, $p_2$, $p_3$ are the positions of each mass in the model.
  - th1, th2 = $\theta_1$, $\theta_2$ are the angles with references in the diagram.
    - th1 is an absolute angle.
    - th2 is a relative angle. 
    - The total absolute angle is th1+th2
  - ell is $\ell$, the length of the stretching spring (distance between p1 and p2)
  - m is the mass of each coil, which is 1 g per coil, or 1e-3 kg/coil
  - L is the length of each coil, which is 0.07 cm = 7e-4 meters
  - g is the gravitational constant, 9.81 m/sec^2

### Remark

- The total length of a typical Slinky is about 7 cm, spread across 98 coils.
- Rounding up to 100 coils gives a convenient estimate of 1 gram per coil, matching the total mass of roughly 100 grams.
- Scaling this two-coil model up to a full Slinky would require repeating this structure 48 more times, alternating between a stretching spring and a bending spring with mass nodes in between.
  
Writing out the symbolic equations for all 98 coils would be extremely tedious. A real engineer would automate this process using:
- **Symbolic modeling tools** (e.g., SymPy, Symbolics.jl), or  
- A **CAD environment** that exports a URDF (Unified Robot Description Format) file, as discussed in the textbook.

This problem will give you a foundation for thinking about energy contributions in spring-mass systems like the Slinky.

In [ ]:
using Symbolics

# This is for a planr model of two coils of a SLINKY

# For the variables, see the schematic 

# generalized coordinates are: x, y, th1, ell, th2
# generalized velocities are: dx, dy, dth1, dell, dth2

# parameters are: 
# g, the gravity constant
# m, the mass of each coil
# k, the spring constant, which for simplicity, we are taking to be 
#    the same for stretching and for bending
# ell0 the rest position of the stretching coil is ell = ell0
# th0 the rest position of the bending coil is th =  0

# The potential enery of a linear spring is (1/2)*k*(ell - ell0)^2 for a stretching spring
# and for a bending spring is (1/2)*k*(th - th0)^2

@Symbolics.variables x y th1 ell th2 dx dy dth1 dell dth2 g m L k ell0

# Form vectors for use in Jacobians
q = [x, y, th1, ell, th2]
dq = [dx, dy, dth1, dell, dth2]

# Kinematics (i.e., positions) to get you started
p0 = [x, y]
p1 = p0 + L*[cos(th1), sin(th1)]
p2 = p1 + (ell + L)*[cos(th1), sin(th1)]

# Fill in the position of the third mass. Remember to use 
# absolute angles as in the textbook
# p3 = p2 + ???

###
### YOUR CODE HERE
###

# Potential Energy terms to get you started
PEmass = g*m*p0[2] + g*m*p1[2] 
PEspring = (1/2)*k*(ell-ell0)^2

# Fill in the PE terms for the other two masses and the bending spring
# PEmass = PEmass + ???
# PEspring = PEspring + ???

###
### YOUR CODE HERE
###

# Sum the individual potential energies to get total Potential Energy
PE = PEmass + PEspring
PE = Symbolics.simplify(PE)
println("PE = ", PE)

# Velocity vectors via the Jacobian!
v0 = Symbolics.jacobian(p0, q) * dq
v1 = Symbolics.jacobian(p1, q) * dq

# fill in the formulas for the velocities of the masses at p2 and p3
# v2 = ??
# v3 = ??

###
### YOUR CODE HERE
###


# Kinetic Energy of each mass on the Robot
KE0 = 0.5 * m * sum(v0.*v0) 
KE1 = 0.5 * m * sum(v1.*v1) 
KE2 = 0.5 * m * sum(v2.*v2)
KE3 = 0.5 * m * sum(v3.*v3)

# Sum the individual KEs to get the total Kinetic Energy
KE = KE0 + KE1 + KE2 + KE3
KE = Symbolics.simplify(KE)
println("\nKE = ", KE)

# Define the Lagrangian
# Lagrangian = 
###
### YOUR CODE HERE
###
println("\nSlinkyLagrangian = ", SlinkyLagrangian)

In [ ]:
# Friendly Check

# Run Tests
test1 = [L*cos(th1 + th2), L*sin(th1 + th2)]
test2 = 0.5*k*(th2^2)
test3 = g*m*(3y + L*sin(th1 + th2) + 2(ell + 2L)*sin(th1))
test4 = [dx + dell*cos(th1) + dth1*(-L*sin(th1) - (L + ell)*sin(th1) - L*sin(th1 + th2)) - L*dth2*sin(th1 + th2), 
   dy + dell*sin(th1) + dth1*(L*cos(th1) + L*cos(th1 + th2) + (L + ell)*cos(th1)) + L*dth2*cos(th1 + th2)]

if iszero(Symbolics.simplify(p3[1] - p2[1] - test1[1]))
    println("The x-component of p3 is correct.")
else
    println("The x-component of p3 is incorrect.")
end

if iszero(Symbolics.simplify(p3[2] - p2[2] - test1[2]))
    println("The y-component of p3 is correct.")
else
    println("The y-component of p3 is incorrect.")
end

if iszero(Symbolics.simplify(PEspring - (1/2)*k*(ell-ell0)^2 - test2))
    println("The total PE due to the springs is correct.")
else
    println("The total PE due to the springs is incorrect.")
end

if iszero(Symbolics.simplify(PEmass - g*m*p1[2] - test3))
    println("The total PE due to the masses is correct.")
else
    println("The total PE due to the masses is incorrect.")
end

if iszero(Symbolics.simplify(v3[1] - test4[1]))
    println("The x-component of v3 is correct.")
else
    println("The x-component of v3 is incorrect.")
end

if iszero(Symbolics.simplify(v3[2] - test4[2]))
    println("The y-component of v3 is correct.")
else
    println("The y-component of v3 is incorrect.")
end

println("\nIf you passed all of these tests, it's hard to see where you could
have made a mistake, but someone will surely prove us wrong!
We are grading PE and KE.")

In [ ]:
# ========== PROBLEM 4 SAMPLE SOLUTION ==========
# You can use this as a reference to check your answer.

### 4 BEGIN SOLUTION 
using Symbolics
@Symbolics.variables x y th1 ell th2 dx dy dth1 dell dth2 g m L k ell0
q = [x, y, th1, ell, th2]
dq = [dx, dy, dth1, dell, dth2]

# Kinematics
p0 = [x, y]
p1 = p0 + L*[cos(th1), sin(th1)]
p2 = p1 + (ell + L)*[cos(th1), sin(th1)]
p3 = p2 + L*[cos(th1+th2), sin(th1+th2)]

# Potential Energy
PEmass = g*m*p0[2] + g*m*p1[2] +  g*m*p2[2] + g*m*p3[2] 
PEspring = (1/2)*k*(ell-ell0)^2 + (1/2)*k*th2^2

PE = PEmass + PEspring
PE = Symbolics.simplify(PE)
println("PE = ", PE)

# Velocity vectors via the Jacobian!
v0 = Symbolics.jacobian(p0, q) * dq
v1 = Symbolics.jacobian(p1, q) * dq
v2 = Symbolics.jacobian(p2, q) * dq
v3 = Symbolics.jacobian(p3, q) * dq

# Kinetic Energy of each mass on the Robot
KE0 = 0.5 * m * sum(v0.*v0) 
KE1 = 0.5 * m * sum(v1.*v1) 
KE2 = 0.5 * m * sum(v2.*v2)
KE3 = 0.5 * m * sum(v3.*v3)

# Sum the individual KEs to get the total Kinetic Energy
KE = KE0 + KE1 + KE2 + KE3
KE = Symbolics.simplify(KE)
println("\nKE = ", KE)

# Define the Lagrangian
SlinkyLagrangian = KE - PE
println("\nSlinkyLagrangian = ", SlinkyLagrangian)
### 4 END SOLUTION

In [ ]:
# ====================================================
# 🔒 GRADER CELL — Problem 4 (Do Not Edit)
# ====================================================
# This cell required previous Friendly checks to be run first.
# Please run this cell before going to the next problem to avoid variable errors.
PEanswer = k*(0.5(th2^2) + 0.5((ell - ell0)^2)) + g*m*(4y + (5L + 2ell)*sin(th1) + L*sin(th1 + th2))
KEanswer = m*(0.5(dx^2) + 0.5(dy^2) + 0.5((dy + L*dth1*cos(th1))^2) + 
    0.5((dx - L*dth1*sin(th1))^2) + 0.5((dx + dell*cos(th1) + dth1*(-L*sin(th1) - (L + ell)*sin(th1)))^2) + 
    0.5((dy + dell*sin(th1) + dth1*(ell + 2L)*cos(th1))^2) + 0.5((dy + dell*sin(th1) + 
            dth1*((ell + 2L)*cos(th1) + L*cos(th1 + th2)) + L*dth2*cos(th1 + th2))^2) + 
    0.5((dx + dth1*(L*(-sin(th1) - sin(th1 + th2)) - (L + ell)*sin(th1)) + dell*cos(th1) - L*dth2*sin(th1 + th2))^2))

point_4A = iszero(Symbolics.simplify(PE - PEanswer)) ? 1 : 0
point_4B = iszero(Symbolics.simplify(KE - KEanswer)) ? 1 : 0
total_score_4 = point_4A + point_4B

# Show score
println("Problem 4 Score: $total_score_4 / 2")

# Problem 5: Lagrange's Equations of Motion and the Robot Equations

<br>

<p align="center">
  <img src="data/HW06LagangeEquationsMotion.png", width="90%">
</p>
<br>

<p align="center">
  <img src="data/HW06RobotEquations.png", width="90%">
</p>
<br>

<p align="center">
  <img src="data/HW06MassInertiaMatrix.png", width="90%">
</p>
<br>

In the following cell, we derive the `D and G terms in the Robot Equations` based on their definition in the attached images (C term uses advanced concept, so it's already given below). Real engineers never compute these terms by hand. They use software. 

In [ ]:
using Symbolics

# Define D and G terms
D = Symbolics.hessian(KE, dq) # Strange name explained in the textbook
G = Symbolics.gradient(PE, q)

# Compute C (advanced, uses Christoffel symbols)
n = length(q)
C = Array{Num}(undef, n, n) # Num is Symbolic Type 
                             
for k in 1:n
    for j in 1:n
        C[k, j] = 0
        for i in 1:n
            C[k, j] += 0.5 * (Symbolics.gradient(D[k, j], q)[i] + 
            Symbolics.gradient(D[k, i], q)[j] - Symbolics.gradient(D[i, j], q)[k]) * dq[i]
        end
    end
end

In [ ]:
# Run me to display the terms in the Robot equations
println("You have computed the terms in a massive 5 degree of freedom model, larger than 
    anything attempted in ME 240. To derive this by hand would be insanely tedious.\n")


D = Symbolics.simplify.(D)
println("\nMass-inertia Matrix")
display(D)

C = Symbolics.simplify.(C)
println("\nCoriolis and Centrifugal Matrix")
display(C)

println("\nGravity Vector")
display(G)

In [ ]:
# Run me to place all of these calculations in a Julia function

include("../data/HW06SymbolicUtilities.jl")

# Create a function that computes all terms
# in the Robot Equations
fcn_name = "dyn_mod_2CoilSlinky"
modelParamString = "g, m, L, k, ell0 = modelParametersSlinky()"
# Need to include line break command \n
variableNamesString = "x, y, th1, ell, th2 = q \ndx, dy, dth1, dell, dth2 = dq"
writeEOM(fcn_name, D, C, G, modelParamString,variableNamesString)

# bring the model into the workspace for later use
include("dyn_mod_2CoilSlinky.jl") 

In [ ]:
# Run me to execute your new function

# bring the model into the workspace for later use
include("dyn_mod_2CoilSlinky.jl") 

# Include utilities where the parameter values are also defined
include("../data/HW06SymbolicUtilities.jl")

# bring the parameter values into the workspace
g, m, L, k, ell0 = modelParametersSlinky()

# x, y, th1, ell, th2
qValues = [0; 0; -pi/2; ell0; pi/8]
dqValues = [1; -1; 5; 5; 5]

F = dyn_mod_2CoilSlinky(qValues, dqValues)

display(F.D)
display(F.C)
display(F.G)

In [ ]:
# ======= Final Score Summary ======= #
# You have to run all previous grader cells to get the final score summary.
total_score = total_score_1 + total_score_2 + total_score_3 + total_score_4

println("🎉🎯 Final Score Summary 🎯🎉")
println("────────────────────────────")
println("Problem 1: $total_score_1")
println("Problem 2: $total_score_2")
println("Problem 3: $total_score_3")
println("Problem 4: $total_score_4")
println("────────────────────────────")
println("🏆 Total Score: $total_score / 10.5")

# (Optional Example) How to Formulate the Flag Pole Problem in JuMP

JuMP computes optimal solutions (minimization or maximization) subject to equality and inequality constraints.

### Remark
This is how a professional would formulate a problem of minimizing a cost subject to constraints in the context of flag bearing.

In [ ]:
using JuMP, Ipopt, LinearAlgebra, Plots
gr()  # Set GR as the backend for Plots

include("../data/HW06JointTorque_utilities.jl")
windX = -10.0  # m/s blowing east to west

# The cost or objective will be the sum of torque squared, 
# as we did before. 
tau(q) = jointTorques(q, windX)
cost(q...) = sum(tau(q) .* tau(q))
# The markdown cell following this cell explains the 
# three dots (called a splat) operation. For now, ignore it.

# Initial joint angles
q0 = [-90; 90; 100] * pi / 180

# Set up the JuMP model
model = Model(Ipopt.Optimizer)
set_optimizer_attribute(model, "max_iter", Int(1e6)) # Adjust max number iterations
set_optimizer_attribute(model, "print_level", 3)  # Adjust verbosity

# let JuMP know about a cost function that is defined outside of JuMP
register(model, :cost, 3, cost; autodiff = true) 

# Define the variables with angles limited to the circle and 
# with initial values
@variable(model, -pi <= q[i=1:3] <= pi, start = q0[i])

# Define the objective function
@NLobjective(model, Min, cost(q[1], q[2], q[3]))

# Add your favorite equality constraints
if true
    @constraint(model, q[1] + q[2] == 0.0)
    @constraint(model, q[1] + q[2] + q[3] - 9*pi/16.0 == 0.0)
else
    @constraint(model, q[1] + pi/2 == 0.0)
    @constraint(model, q[1] + q[2] + q[3]-1.05*pi/2 == 0)
end


# Optimize the model
optimize!(model)

# Extract the optimal joint angles
q_optimal = value.(q)
tau_optimal = tau(q_optimal)

# Output the result
println("\nThe cost at the initial joint angles is: ", cost(q0...))
println("The cost at the optimal joint angles is: ", cost(q_optimal...))

println("\nThe optimal joint angles that minimize the cost are: ", q_optimal, "  radians")
println("The optimal joint angles that minimize the cost are: ", q_optimal * 180 / pi, "  degrees\n")

println("The optimal torques are:   ", tau_optimal, "  Newtons")
println("The original torques were: ", tau(q0), "  Newtons")

# Initialize the plot
fig = plot(; legend = :topleft, aspect_ratio = :equal)
plot_armFlagBearer(q_optimal)
display(fig)


println("The optimal solution seems to be adjusting the Center of Mass of the Arm + Flagpole
    to be as closely aligned with the shoulder joint as the constraints allow.")

println("\nNotice that the optimal shoulder torque is zero, with the elbow and wrist bearing 
    the load of the flag pole. Biomechanically, the wrist and elbow are weaker and more easily
    damaged than the shoulder joint. Hence, the posture that minimizes our cost function, sum of
    torque squared, is giving us a solution that seems wrong. What this means is that our cost 
    function was not chosen correctly. In other words, our working hypothesis: 'joint positions 
    that minimize the sum of the squared joint torques will maximize overall joint health' was
    likely INCORRECT and needs to be modified before we submit our IRB. This is what mathematical
    analysis is for: sorting out ideas before testing them on hardware or organic-ware.")


## Explanation for `cost(q...)`

In Julia, the `...` syntax, known as the "splat" operator, is used to unpack the elements of an iterable (such as an array or tuple) and pass them as individual arguments to a function. This is useful when you want to call a function that takes multiple arguments, but you have them stored in a single array or tuple.

In the function definition:

```julia
cost(q...) = sum(tau(q) .* tau(q))
```

- `q...` is a way to indicate that the function `cost` takes a variable number of arguments. When `q` is passed to the function, it can be unpacked into individual elements.
- Inside the function, `q` is treated as a tuple of arguments.

How it works step-by-step:

1. The function `cost` can accept a variable number of arguments. For example, you could call `cost(1, 2, 3)` or `cost(1, 2, 3, 4)`, and `q...` will handle them appropriately.
2. When you call `cost(q...)` with `q` as an array or tuple, the elements of `q` are unpacked and passed as individual arguments to the function `tau`.
3. Inside the `cost` function, `tau(q)` is called with the unpacked elements of `q`.

### Example

Consider a simple example to illustrate the `...` operator:

```julia
# Define a function that takes a variable number of arguments
function example_function(a, b, c)
    return a + b + c
end

# Define another function that unpacks its arguments
function wrapper_function(args...)
    return example_function(args...)
end

# Call the wrapper function with a tuple of arguments
args = (1, 2, 3)
println(wrapper_function(args...))  # Output: 6
```

In this example, `wrapper_function(args...)` unpacks the tuple `args` and passes the individual elements to `example_function`.

### Applying to Your Code

In your code:

```julia
tau(q) = jointTorques(q, windX)
cost(q...) = sum(tau(q) .* tau(q))
```

- `tau(q)` computes the torques given the joint angles `q` and wind speed `windX`.
- `cost(q...)` defines a function that computes the sum of the squared torques, where `q...` unpacks the joint angles so that `tau` receives them as individual arguments.

This allows the function `cost` to be called with a variable number of arguments, making it flexible and compatible with optimization routines that might handle the joint angles as a single array or tuple.


<p align="center" style="font-size:48px;"><strong>The End! </strong></p>